## PUNTO # 3

[35p] Recuperación ranqueada y vectorización de documentos (RRDV)


La recuperación ranqueada y vectorización de documentos (RRDV) es un enfoque dentro de la recuperación de información que implica dos procesos principales:

Vectorización de documentos: Esto se refiere a la representación de textos (tanto documentos como queries) como vectores en un espacio multidimensional. Cada dimensión del espacio representa un término del vocabulario total (es decir, todas las palabras únicas en el corpus). Las técnicas comunes para vectorizar documentos incluyen la ponderación de términos utilizando esquemas como TF-IDF (Term Frequency-Inverse Document Frequency), donde se captura la importancia de cada término en el contexto de un documento específico y en el corpus en general.

Recuperación ranqueada: Después de que los documentos y queries han sido vectorizados, la recuperación ranqueada implica la comparación de estos vectores para determinar la relevancia de cada documento en relación con una query específica. Los documentos son entonces "ranqueados" o clasificados en función de su similitud con la query, utilizando medidas como la similitud coseno. Los documentos más relevantes (aquellos con mayor similitud) aparecen en los primeros lugares del ranking, facilitando el acceso a la información más pertinente.

Importación de librerías requeridas

In [1]:
import nltk
import numpy as np
import pandas as pd

Definición de paths requeridos para la correcta ejecución del ejercicio

In [2]:
##################
## Paths para apuntar a la data
###################
path_docs = 'C:/Users/DELL/Documents/docs-raw-texts' #path de los documentos
path_queries = 'C:/Users/DELL/Documents/queries-raw-texts' #path de las queries
ground_truth_path = 'C:/Users/DELL/Documents/relevance-judgments.tsv'  #path de lectura del ground-truth
salidaFile = "RRDV-consultas_resultados.tsv"  #path para exportación de resultados

# Fase de preprocesamiento del corpus.

La siguiente implementación define una clase de procesamiento de texto que realiza el pre-procesamiento de documentos de texto. Los pasos incluyen:

1. Tokenización: Divide el texto en palabras o tokens.
2. Eliminación de palabras vacías: Elimina palabras comunes que no aportan significado (como "the", "and").
3. Lematización: Reduce las palabras a su forma base o raíz.
4. Creación de un vocabulario: Genera un conjunto de términos únicos presentes en los documentos.

El código organiza estos procesos en métodos dentro de la clase, y luego ejecuta el preprocesamiento sobre un conjunto de documentos, almacenando los resultados para su posterior uso en tareas de análisis de texto.

In [3]:
#  Pasos de preprocesamiento: para los siguientes puntos,
#  debe preprocesar documentos y consultas mediante tokenización a nivel de palabra,
#  eliminación de palabras vacías, normalización y stemming

import os
from pathlib import Path
import re

class procesamientotexto:

    def __init__(self,path_dir):
        self.path = path_dir
        self.tokens_doc = {} 
        self.word_tok_nltk_es_sw = {}
        self.nltk_lemmaList = {}

    def tokenizacion(self):
        tokenizer = nltk.RegexpTokenizer(r'\w+')
        for doc in os.listdir(self.path):
            path = os.path.join(self.path,doc)
            content_archivo = open(path,encoding='utf8').read()
            texto = re.match('[\w\W]+<raw><!\[CDATA\[(?P<texto>(.|\n|\s|\s)+)\]\]></raw>',content_archivo).groupdict()['texto'].lower()
            # self.tokens_doc[doc] = nltk.word_tokenize(texto,preserve_line=True)
            self.tokens_doc[doc] = tokenizer.tokenize(texto)
            
        return self.tokens_doc
    
    def stopwords(self):
        nltk_stop_words_es = set(nltk.corpus.stopwords.words('english'))
        for name_doc,doc in self.tokens_doc.items():
            self.word_tok_nltk_es_sw[name_doc] = [token for token in doc if token not in nltk_stop_words_es ]
        return self.word_tok_nltk_es_sw
    
    def stemming(self): 
        wordnet_lemmatizer = nltk.stem.WordNetLemmatizer()
        index = 0
        for name_doc, doc in self.word_tok_nltk_es_sw.items():
            index += 1
            term_count = {}
            lemmatized_text = []
            for word in doc:
                lemmatized_word = wordnet_lemmatizer.lemmatize(word)
                lemmatized_text.append(lemmatized_word)
                if lemmatized_word in term_count:
                    term_count[lemmatized_word] += 1
                else:
                    term_count[lemmatized_word] = 1
            self.nltk_lemmaList[name_doc] = {
                'index': int(re.match('wes2015.(d|q)(?P<num>\d+).naf', name_doc).groupdict()['num']),
                'text': lemmatized_text,
                'term_count': term_count
            }
        return self.nltk_lemmaList
    
    def dicterminos(self):
        dic = set([])
        for doc in self.nltk_lemmaList.values():
            dic = dic.union(set(doc['text'])) 
        return dic

    
text_process = procesamientotexto(path_docs)
doc_tokens = text_process.tokenizacion()
word_tok_nltk_es_sw = text_process.stopwords()
nltk_lemmaList = text_process.stemming()  # los textos lemmatizados
dicterminos = text_process.dicterminos()  # vocabulario

# Número total de documentos
N = len(nltk_lemmaList)

print(nltk_lemmaList['wes2015.d001.naf'])
print(dicterminos)
print("Documentos: ", N)


{'index': 1, 'text': ['william', 'beaumont', 'human', 'digestion', 'william', 'beaumont', 'physiology', 'digestion', 'image', 'source', 'november', '21', '1785', 'u', 'american', 'surgeon', 'william', 'beaumont', 'born', 'became', 'best', 'known', 'father', 'gastric', 'physiology', 'following', 'research', 'human', 'digestion', 'william', 'beaumont', 'born', 'lebanon', 'connecticut', 'became', 'physician', 'served', 'surgeon', 'mate', 'army', 'war', '1812', 'opened', 'private', 'practice', 'plattsburgh', 'new', 'york', 'rejoined', 'army', 'surgeon', '1819', 'beaumont', 'stationed', 'fort', 'mackinac', 'mackinac', 'island', 'michigan', 'early', '1820s', 'existed', 'protect', 'interest', 'american', 'fur', 'company', 'fort', 'became', 'refuge', 'wounded', '19', 'year', 'old', 'french', 'canadian', 'fur', 'trader', 'named', 'alexis', 'st', 'martin', 'shotgun', 'went', 'accident', 'american', 'fur', 'company', 'store', 'close', 'range', 'june', '6th', '1822', 'st', 'martin', 'wound', 'quit

# Implementación del indice invertido

Esta implementación crea un índice invertido a partir de un conjunto de documentos lematizados, a través de una estructura de datos que asocia cada término del vocabulario con los documentos en los que aparece. Los componentes de esta implementación, són:

1. Inicialización del índice: Para cada término del vocabulario, se crea una entrada en el índice invertido con una lista vacía para los documentos y un contador de frecuencia.
2. Llenado del índice: Se recorren todos los documentos. Para cada término presente en un documento, se añade el identificador del documento a la lista correspondiente en el índice y se incrementa el contador de frecuencia.
3. Resultado: El índice invertido permite saber en qué documentos aparece cada término y cuántos documentos lo contienen, facilitando la búsqueda de información en el corpus.

In [4]:
# Implementación del índice invertido usando 
# los 331 documentos en el conjunto de datos.

def indiceinvertido(doc_lemalist: dict,terminos:dict):
    indiceinvertido = {}
    for termino in terminos: 
         indiceinvertido[termino] = {'IDdocs':[],'len':0}
  
    for documento in doc_lemalist.values():  
        set_texto = set(documento['text'])
        for termino in set_texto: 
            indiceinvertido[termino]['IDdocs'].append(documento['index'])
            indiceinvertido[termino]['len'] +=1 
    return indiceinvertido


list_indiceinvertido = indiceinvertido(nltk_lemmaList,dicterminos)
print(list_indiceinvertido)

{'half': {'IDdocs': [6, 12, 27, 32, 39, 44, 49, 60, 78, 101, 113, 125, 127, 139, 162, 167, 176, 184, 201, 222, 256, 268, 272, 281, 315, 317], 'len': 26}, 'precocious': {'IDdocs': [260, 263], 'len': 2}, 'swedberg': {'IDdocs': [294], 'len': 1}, 'jon': {'IDdocs': [63], 'len': 1}, 'maneuver': {'IDdocs': [62], 'len': 1}, 'archipelago': {'IDdocs': [56, 239], 'len': 2}, 'differed': {'IDdocs': [32, 67, 134, 189, 312], 'len': 5}, 'unitarian': {'IDdocs': [88, 248], 'len': 2}, 'asphyxia': {'IDdocs': [84], 'len': 1}, 'hoped': {'IDdocs': [20, 96, 102, 123, 227, 281], 'len': 6}, 'rebuilt': {'IDdocs': [58], 'len': 1}, 'ballerina': {'IDdocs': [101], 'len': 1}, 'refractor': {'IDdocs': [140, 265], 'len': 2}, 'wagon': {'IDdocs': [128, 138], 'len': 2}, 'hemoptysis': {'IDdocs': [293], 'len': 1}, 'laborious': {'IDdocs': [165], 'len': 1}, 'miller': {'IDdocs': [52, 257], 'len': 2}, 'battle': {'IDdocs': [12, 18, 24, 25, 33, 49, 55, 71, 90, 139, 168, 177, 185, 241, 302], 'len': 15}, 'cryptologist': {'IDdocs': [

# Creación de una representación vectorial a partir de la generación del indice invertido, como fase previa de la fase de procesamiento de similitud coseno
Este código calcula la representación TF-IDF de documentos. Recorre términos y documentos, computando TF-IDF para cada uno. Es preciso pero puede ser costoso en tiempo para grandes colecciones, los pasos más destacados de la implementación son los siguientes:
1. Cálculo de TF: Se calcula la frecuencia de cada término en un documento, ajustada logarítmicamente.
2. Cálculo de IDF: Se ajusta la importancia del término según la cantidad de documentos en los que aparece.
3. Cálculo de TF-IDF: Multiplicando TF por IDF, se obtiene un valor ponderado para cada término en el documento.

In [5]:
# [10p] Cree una función que, a partir del índice invertido, 
# cree la representación vectorial ponderada tf.idf de un documento o consulta. 
# Describa en detalle su estrategia, ¿es eficiente? ¿por qué si, por qué no?

import numpy as np

def calcular_tf_idf(nltk_lemmaList, dicterminos, list_indiceinvertido, N):
    tf_idf = {}

    for doc_name, doc_data in nltk_lemmaList.items():
        tf_idf[doc_name] = {}
        
        for term in dicterminos:
            tf = doc_data['term_count'].get(term, 0)
            df = list_indiceinvertido[term]['len']

            if tf > 0 and df > 0:
                # Cálculo de TF y DF según las fórmulas proporcionadas
                tf_value = np.log10(1 + tf)
                df_value = np.log10(N / df)
                
                # Cálculo de TF-IDF
                tf_idf[doc_name][term] = tf_value * df_value
            else:
                tf_idf[doc_name][term] = 0.0

    return tf_idf

# Calcular la representación TF-IDF
tf_idf_representation = calcular_tf_idf(nltk_lemmaList, dicterminos, list_indiceinvertido, N)
print(tf_idf_representation['wes2015.d001.naf'])
print(tf_idf_representation['wes2015.d002.naf'])

{'half': 0.0, 'precocious': 0.0, 'swedberg': 0.0, 'jon': 0.0, 'maneuver': 0.0, 'archipelago': 0.0, 'differed': 0.0, 'unitarian': 0.0, 'asphyxia': 0.0, 'hoped': 0.0, 'rebuilt': 0.0, 'ballerina': 0.0, 'refractor': 0.0, 'wagon': 0.0, 'hemoptysis': 0.0, 'laborious': 0.0, 'miller': 0.0, 'battle': 0.0, 'cryptologist': 0.0, 'refers': 0.0, 'construction': 0.0, 'armour': 0.0, 'film': 0.0, 'murcia': 0.0, 'stabilize': 0.0, 'interrogator': 0.0, 'misinterpreted': 0.0, 'moldavia': 0.0, 'annihilation': 0.0, 'zum': 0.0, 'combinatorics': 0.0, 'instinctively': 0.0, 'hindered': 0.0, 'heroine': 0.0, 'ou': 0.0, 'performer': 0.0, 'reinforced': 0.0, 'scheele': 0.0, 'parsley': 0.0, 'vase': 0.0, 'caloun': 0.0, 'finishing': 0.0, 'reversible': 0.0, 'weaker': 0.0, 'readmitted': 0.0, 'second': 0.0, 'listed': 0.0, 'distinctive': 0.0, 'attenborough': 0.0, 'nombres': 0.0, 'diatribae': 0.0, 'encyclopédie': 0.0, 'faster': 0.0, 'interconnected': 0.0, 'oort': 0.0, 'lancashire': 0.0, 'newfoundland': 0.0, 'cambria': 0.0, '

Con la representación vectorial generada, esta sección lo representa a través de un DataFrame para facilidad de lectura.


In [6]:
# Función para crear un dataframe que permita visualizar los datos para cada termino, 
# y su ponderación TF-IDF en un dataFrame para cada documento

def generar_vector_tf_idf_df(tf_idf_representation, doc_name, dicterminos):
    data = [{'Término': term, 'TF-IDF': tf_idf_representation[doc_name].get(term, 0.0)} for term in dicterminos]
    df = pd.DataFrame(data)
    return df

# Ejemplo de uso para el documento 'wes2015.d001.naf'
doc_name = 'wes2015.d001.naf'
df_vector_tf_idf = generar_vector_tf_idf_df(tf_idf_representation, doc_name, dicterminos)
print(df_vector_tf_idf)

            Término    TF-IDF
0              half  0.000000
1        precocious  0.000000
2          swedberg  0.000000
3               jon  0.000000
4          maneuver  0.000000
...             ...       ...
16834       regnant  0.000000
16835  kaiserswerth  0.000000
16836      subgroup  0.000000
16837    larchfield  0.000000
16838   plattsburgh  0.758544

[16839 rows x 2 columns]


Esta sección genera un vector de documento del TF-ID

In [7]:
# Función de apoyo para crear vector de documento TF-IDF

def generar_vector_tf_idf(tf_idf_representation, doc_name, dicterminos):
    vector_tf_idf = []   
    for term in dicterminos:
        tf_idf_value = tf_idf_representation[doc_name].get(term, 0.0)
        vector_tf_idf.append(tf_idf_value) 
    return vector_tf_idf

# Ejemplo de uso para el documento 'wes2015.d001.naf'
doc_name = 'wes2015.d001.naf'
vector_tf_idf = generar_vector_tf_idf(tf_idf_representation, doc_name, dicterminos)
print(vector_tf_idf)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.758543810040283, 0.0, 0.0, 2.806945642623135, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

# Procesamiento de la similitud coseno entre dos vectores con la representación TF-IDF de dos documentos
Este código calcula la similitud del coseno entre dos documentos usando sus representaciones TF-IDF. Genera los vectores de términos de los documentos, calcula el producto punto y divide por el producto de las normas de los vectores. El rango de la similitud del coseno tiene un rango de valores entre -1 y 1. Un valor de 1 indica que los vectores son completamente similares (paralelos), 0 indica que no tienen correlación (perpendiculares), y -1 indica que son completamente opuestos (antiparalelos). Sin embargo, cuando se trabaja con TF-IDF, los valores suelen estar entre 0 y 1 porque los pesos de TF-IDF son no negativos.
Para esta implementación, el enfoque para calcular la similitud coseno entre dos documentos representados como vectores se basa en el producto punto y las normas de los vectores. El proceso es el siguiente:

1. Producto punto: Se calcula el producto punto entre los dos vectores que representan los documentos (vector de TF-IDF o cualquier representación vectorial). En código, esto se hace con np.dot(arr1, arr2).
2. Norma de los vectores: Luego, se calcula la norma (longitud) de cada vector mediante np.linalg.norm(arr).
3. División: Finalmente, se divide el producto punto entre el producto de las normas de ambos vectores, lo que da el valor de la similitud coseno

In [8]:
# [10p] Cree una función que reciba dos vectores de documentos y calcule la similitud del coseno.

from numpy.linalg import norm

def simil_Coseno(doc1, doc2, tf_idf_representation, dicterminos):
    
    vector1 = generar_vector_tf_idf(tf_idf_representation, doc1, dicterminos)
    vector2 = generar_vector_tf_idf(tf_idf_representation, doc2, dicterminos)
    arr1 = np.array(vector1)
    arr2 = np.array(vector2)
    
    if len(arr1)==len(arr2):
        coseno = np.dot(arr1,arr2)/(norm(arr1)*norm(arr2))
    else:
        coseno = "error de comparación"   
    return coseno
    
# Ejemplo de uso
DOC1_name = 'wes2015.d001.naf'
DOC2_name = 'wes2015.d002.naf'
CosenoSimil = simil_Coseno(DOC1_name, DOC2_name, tf_idf_representation, dicterminos)
print("Similitud Coseno:\n", CosenoSimil)

Similitud Coseno:
 0.012657298971615048


# Recuperación de los documentos clasificados
Esta sección, procede a tomar las queries mediante el archivo externo, realizando a este conjunto de datos un preprocesamiento de texto, y posterioremente procesar para generar una representación vectorial del TF-IDF de cada documento.
Esta implementación realiza la recuperación de documentos en un corpus utilizando la similitud coseno. Primero, se procesan las consultas aplicando tokenización, eliminación de stopwords y lematización. Luego, se genera la representación TF-IDF de las consultas y documentos. Para cada consulta, se calculan las similitudes coseno entre el vector de la consulta y los vectores de los documentos. Finalmente, los documentos se ordenan según su similitud con la consulta y se almacenan en un archivo de salida, mostrando solo aquellos con similitud mayor a 0.


In [9]:
# [5p] Para cada una de las 35 consultas en el conjunto de datos, 
# recupere los documentos clasificados -ordenados por el puntaje de similitud del coseno-
# (incluya solo los documentos con un puntaje superior a 0 para una consulta determinada). 
# Escriba un archivo (RRDV-consultas_resultados) con los resultados siguiendo el siguiente formato:
# q01 dXX: cos_simi(q01,dXX),dYY: cos_simi(q01, dYY),dZZ: cos_simi(q01,dZZ)…

proc_querys = procesamientotexto(path_queries)
tokens_querys = proc_querys.tokenizacion()
tokens_querys_sw = proc_querys.stopwords()
terminos_querys = proc_querys.stemming()
print(terminos_querys)
Nqueries = len(terminos_querys)
print(Nqueries)

# Calcular la representación TF-IDF para los queries
doc_query = 'wes2015.q01.naf'
tf_idf_rep_queries = calcular_tf_idf(terminos_querys, dicterminos, list_indiceinvertido, N)
# Ejemplo de uso para el documento 'wes2015.q01.naf'
df_vectorQuery_tf_idf = generar_vector_tf_idf_df(tf_idf_rep_queries, doc_query, dicterminos)
df_vectorQuery_tf_idf


{'wes2015.q01.naf': {'index': 1, 'text': ['fabrication', 'music', 'instrument'], 'term_count': {'fabrication': 1, 'music': 1, 'instrument': 1}}, 'wes2015.q02.naf': {'index': 2, 'text': ['famous', 'german', 'poetry'], 'term_count': {'famous': 1, 'german': 1, 'poetry': 1}}, 'wes2015.q03.naf': {'index': 3, 'text': ['romanticism'], 'term_count': {'romanticism': 1}}, 'wes2015.q04.naf': {'index': 4, 'text': ['university', 'edinburgh', 'research'], 'term_count': {'university': 1, 'edinburgh': 1, 'research': 1}}, 'wes2015.q06.naf': {'index': 6, 'text': ['bridge', 'construction'], 'term_count': {'bridge': 1, 'construction': 1}}, 'wes2015.q07.naf': {'index': 7, 'text': ['walk', 'fame', 'star'], 'term_count': {'walk': 1, 'fame': 1, 'star': 1}}, 'wes2015.q08.naf': {'index': 8, 'text': ['scientist', 'worked', 'atomic', 'bomb'], 'term_count': {'scientist': 1, 'worked': 1, 'atomic': 1, 'bomb': 1}}, 'wes2015.q09.naf': {'index': 9, 'text': ['invention', 'internet'], 'term_count': {'invention': 1, 'inte

,Término,TF-IDF
0,half,0.0
1,precocious,0.0
2,swedberg,0.0
3,jon,0.0
4,maneuver,0.0
...,...,...
16834,regnant,0.0
16835,kaiserswerth,0.0
16836,subgroup,0.0
16837,larchfield,0.0


# Calculo de la similitud coseno entre consultas y documentos, ordena los resultados, y los guarda en un archivo y en un DataFrame.
En esta sección, ya se procede a realizar el procesamiento de información entre la representación vectorial del corpus, contra la representacióon vectorial de los queries, mediante la tecnica de similitud coseno, al final de esta sección se generará una representación que se exporta a un archivo externo, y también cuenta con una representación en memoria mediante un DataFrame para proximas operaciones

In [18]:
# Desarrollo para evaluar el conjunto de queries con el conjunto de documentos

def simil_Coseno(vector1, vector2):
    arr1 = np.array(vector1)
    arr2 = np.array(vector2)
    
    if len(arr1) == len(arr2):
        coseno = np.dot(arr1, arr2) / (norm(arr1) * norm(arr2))
    else:
        coseno = 0 
    return coseno

def generar_similitudes_y_dataframe(tf_idf_representation, tf_idf_rep_queries, dicterminos):
    resultados = []
    data = []  
    
    for q_name, q_vector in tf_idf_rep_queries.items():
        q_id = f"q{int(q_name.split('.')[1][1:]):02d}" 
        similitudes = []
        
        for doc_name, doc_vector in tf_idf_representation.items():
            doc_id = f"d{int(doc_name.split('.')[1][1:]):02d}"  
            coseno_sim = simil_Coseno(
                generar_vector_tf_idf(tf_idf_representation, doc_name, dicterminos),
                generar_vector_tf_idf(tf_idf_rep_queries, q_name, dicterminos)
            )
            if coseno_sim > 0:
                similitudes.append((doc_id, coseno_sim))
        
        similitudes.sort(key=lambda x: x[1], reverse=True)
        similitudes_str = ",".join([f"{doc_id}:{similitud:.4f}" for doc_id, similitud in similitudes])
        
        if similitudes_str:
            resultados.append(f"{q_id}\t{similitudes_str}")  
            data.append({'query_id': q_id, 'docs': similitudes_str})
    
    resultados_df = pd.DataFrame(data)
   
    return resultados, resultados_df

def escribir_resultados(resultados, salida):
    with open(salida, 'w', encoding='utf-8') as file:
        for resultado in resultados:
            file.write(resultado + "\n") 

resultados, resultados_df = generar_similitudes_y_dataframe(tf_idf_representation, tf_idf_rep_queries, dicterminos)
escribir_resultados(resultados, salidaFile)
resultados_df

,query_id,docs
0,q01,"d170:0.0845,d85:0.0752,d254:0.0548,d16:0.0499,..."
1,q02,"d147:0.1211,d149:0.0898,d283:0.0777,d293:0.066..."
2,q03,"d283:0.0727,d291:0.0663,d147:0.0572,d318:0.055..."
3,q04,"d270:0.1238,d19:0.1187,d310:0.1017,d49:0.0987,..."
4,q06,"d329:0.2327,d297:0.2205,d26:0.1538,d29:0.1137,..."
5,q07,"d146:0.1223,d04:0.1066,d289:0.0677,d262:0.0498..."
6,q08,"d110:0.1514,d251:0.1328,d117:0.1232,d108:0.103..."
7,q09,"d198:0.1531,d199:0.1267,d223:0.0968,d177:0.093..."
8,q10,"d231:0.0691,d60:0.0652,d100:0.0597,d36:0.0244,..."
9,q12,"d277:0.1723,d258:0.1296,d176:0.0802,d239:0.077..."


# Tratamiento del "Ground-Truth"
En esta parte del desarrollo propuesto, se procede a importar los datos del "ground-truth", con el fin de evaluar y validar los resultados hasta ahora procesados mediante las metricas de evaluación propuestas.
Es importante contrastar los resultados obtenidos de la similitud coseno con un ground-truth porque este último actúa como referencia objetiva o verdad establecida. Esto permite:
1. Evaluar la precisión: Verificar qué tan bien los documentos recuperados por el modelo reflejan los documentos relevantes establecidos por el ground-truth.
2. Medir la efectividad: Las métricas como Precision, Recall, NDCG y MAP comparan las predicciones del modelo con el ground-truth para cuantificar su rendimiento.
3. Identificar mejoras: Ayuda a detectar las limitaciones del modelo y ajustar sus parámetros para mejorar su rendimiento en tareas de recuperación de información.
4. Validación: Asegura que el modelo generalice bien y no esté sesgado hacia patrones específicos que no reflejan relevancia real en el contexto de las consultas.

In [19]:
# [10p] Evaluación de resultados. Calcule P@M, R@M, NDCG@M por consulta. M es el número de
# documentos relevantes encontrados en el archivo de juicios de relevancia por consulta. Luego calcule MAP
# como una métrica general.

# NOTA I: Para P@M y R@M suponga una escala de relevancia binaria. Los documentos que no se
# encuentran en el archivo “relevance-judgments” NO son relevantes para una consulta determinada.
# NOTA II: Para NDCG@M utilice la escala de relevancia no binaria que se encuentra en el archivo
# “relevance-judgments”.

# PROCESAMIENTO INICIAL DEL GROUND-TRUTH
ground_truth_df = pd.read_csv(ground_truth_path, sep='\t', header=None, names=['query_id', 'doc_id_relevance'])
ground_truth_df['doc_ids'] = ground_truth_df['doc_id_relevance'].apply(lambda x: [doc.split(':')[0] for doc in x.split(',')])
ground_truth_df['relevance'] = ground_truth_df['doc_id_relevance'].apply(lambda x: [int(doc.split(':')[1]) for doc in x.split(',')])
ground_truth_df[['query_id', 'doc_ids', 'relevance']]

,query_id,doc_ids,relevance
0,q01,"[d186, d254, d016]","[4, 5, 5]"
1,q02,"[d136, d139, d143, d283, d228, d164, d318, d29...","[2, 2, 4, 4, 4, 4, 2, 4, 4, 2, 2]"
2,q03,"[d152, d291, d283, d147, d318, d105]","[3, 4, 4, 3, 2, 2]"
3,q04,"[d275, d010, d286, d019, d049, d330, d270]","[3, 3, 2, 2, 2, 2, 3]"
4,q06,"[d069, d233, d257, d297, d026, d329]","[2, 3, 2, 3, 4, 5]"
5,q07,"[d004, d077, d266, d179]","[3, 3, 2, 3]"
6,q08,"[d205, d005, d110, d108, d117, d081, d292, d25...","[2, 4, 4, 3, 3, 2, 2, 5, 3, 3, 2, 2]"
7,q09,"[d205, d199, d198, d223, d217, d177]","[3, 5, 3, 2, 2, 2]"
8,q10,"[d068, d100, d065, d076, d231, d199, d052, d215]","[2, 2, 3, 3, 4, 4, 2, 2]"
9,q12,"[d239, d277, d258, d250]","[4, 4, 3, 4]"


# Calculo de la metrica de evaluación 𝑃@𝑀
Teniendo en memoria la información del "Ground-truth" mediante una represetnación de DataFrame, esto peritte a continuación contrastar la información procesada mediante la metrica de evalaución PRECISION, los resultados son mostrados en un DataFrame para facilidad de lectura.
Este código calcula la precisión 𝑃@𝑀 para cada consulta y la guarda en un DataFrame, comparando los documentos recuperados con los relevantes.
La métrica 𝑃@𝑀 (Precision at M) mide la precisión de los primeros M documentos recuperados en una búsqueda. Se calcula dividiendo el número de documentos relevantes entre los primeros M recuperados.

In [20]:
# Calculo de P@M

def limpiar_identificadores(doc_ids):
    return list(set([doc.replace(')', '').strip() for doc in doc_ids]))

def calcular_precision_p_m(resultados_df, ground_truth_df):
    precision_por_query = {}
    if 'doc_ids' not in resultados_df.columns:
        resultados_df['doc_ids'] = resultados_df['docs'].apply(lambda x: [doc.split(':')[0] for doc in x.split(',')])
    resultados_df['doc_ids'] = resultados_df['doc_ids'].apply(limpiar_identificadores)   
    if 'doc_ids' not in ground_truth_df.columns:
        ground_truth_df['doc_ids'] = ground_truth_df['doc_id_relevance'].apply(lambda x: [doc.split(':')[0] for doc in x.split(',')])
    for _, row in resultados_df.iterrows():
        query_id = row['query_id']
        doc_retrieved = row['doc_ids']   
        M = len(doc_retrieved)     
        ground_truth_row = ground_truth_df[ground_truth_df['query_id'] == query_id]   
        if ground_truth_row.empty:
            precision_por_query[query_id] = 0.0
            continue
        relevancia_docs = ground_truth_row['doc_ids'].values[0]
        relevance_list = [1 if doc in relevancia_docs else 0 for doc in doc_retrieved[:M]]
        precision_p_m = sum(relevance_list) / M if M > 0 else 0.0
        precision_por_query[query_id] = precision_p_m
            
    return precision_por_query

precision_resultados = calcular_precision_p_m(resultados_df, ground_truth_df)
precisionDF = pd.DataFrame(list(precision_resultados.items()), columns=['query_id', 'precision'])
precisionDF

,query_id,precision
0,q01,0.041667
1,q02,0.055838
2,q03,1.000000
3,q04,0.018519
4,q06,0.100000
5,q07,0.020408
6,q08,0.052023
7,q09,0.109091
8,q10,0.020513
9,q12,0.111111


# Calculo de la metrica de evaluación 𝑅@𝑀
De la misma manera, a continuación se realiza el procesamiento de la información para aplicar la metrica de evaluación 𝑅@𝑀 (Recall), los resultados son mostrados en un DataFrame para facilidad de lectura. La métrica 𝑅@𝑀 (Recall at M) mide la proporción de documentos relevantes que han sido recuperados en los primeros M documentos. Se calcula dividiendo el número de documentos relevantes recuperados entre el total de documentos relevantes existentes. 

In [21]:
# Calculo de R@M

def calcular_recall_p_m(resultados_df, ground_truth_df):
    recall_por_query = {}
    if 'doc_ids' not in resultados_df.columns:
        resultados_df['doc_ids'] = resultados_df['docs'].apply(lambda x: [doc.split(':')[0] for doc in x.split(',')])
    if 'doc_ids' not in ground_truth_df.columns:
        ground_truth_df['doc_ids'] = ground_truth_df['doc_id_relevance'].apply(lambda x: [doc.split(':')[0] for doc in x.split(',')])
    for _, row in resultados_df.iterrows():
        query_id = row['query_id']
        doc_retrieved = row['doc_ids'] 
        relevancia_docs = ground_truth_df[ground_truth_df['query_id'] == query_id]['doc_ids'].values[0]
        num_total_relevant_docs = len(relevancia_docs)
        if num_total_relevant_docs == 0:
            recall_por_query[query_id] = 0.0
            continue
        relevance_list = [1 if doc in relevancia_docs else 0 for doc in doc_retrieved]
        recall_p_m = sum(relevance_list) / num_total_relevant_docs
        recall_por_query[query_id] = recall_p_m
    
    return recall_por_query

recall_resultados = calcular_recall_p_m(resultados_df, ground_truth_df)
recallDF = pd.DataFrame(list(recall_resultados.items()), columns=['query_id', 'recall'])
recallDF


,query_id,recall
0,q01,0.666667
1,q02,1.000000
2,q03,1.000000
3,q04,0.571429
4,q06,0.666667
5,q07,0.250000
6,q08,0.750000
7,q09,1.000000
8,q10,0.500000
9,q12,1.000000


# Calculo de la metrica de evaluación 𝑁𝐷𝐶𝐺@𝑀
En la misma proporción, se realiza el procesamiento de la información para aplicar la metrica de evaluación 𝑁𝐷𝐶𝐺@𝑀 (Normalized Discounted Cumulative Gain at M), los resultados son mostrados en un DataFrame para facilidad de lectura. La métrica 𝑁𝐷𝐶𝐺@𝑀 (Normalized Discounted Cumulative Gain) mide la calidad de los resultados recuperados considerando la relevancia y su posición en la lista de resultados. Esta implementación calcula 𝑁𝐷𝐶𝐺@𝑀 para cada consulta y lo almacena en un DataFrame. Utiliza la relevancia de los documentos recuperados y los compara con el orden ideal para cada consulta

In [22]:
# Calculo del NDCG@M

def calcular_dcg(relevancias, M):
    return sum((relevancia / np.log2(idx + 2)) for idx, relevancia in enumerate(relevancias[:M]))

def calcular_ndcg_p_m(resultados_df, ground_truth_df):
    ndcg_por_query = {}
    if 'doc_ids' not in resultados_df.columns:
        resultados_df['doc_ids'] = resultados_df['docs'].apply(lambda x: [doc.split(':')[0] for doc in x.split(',')])
    resultados_df['doc_ids'] = resultados_df['doc_ids'].apply(limpiar_identificadores)
    if 'doc_ids' not in ground_truth_df.columns:
        ground_truth_df['doc_ids'] = ground_truth_df['doc_id_relevance'].apply(lambda x: [doc.split(':')[0] for doc in x.split(',')]) 
    ground_truth_df['relevancias'] = ground_truth_df['doc_id_relevance'].apply(lambda x: [int(doc.split(':')[1]) for doc in x.split(',')])
    for _, row in resultados_df.iterrows():
        query_id = row['query_id']
        doc_retrieved = row['doc_ids'] 
        ground_truth_row = ground_truth_df[ground_truth_df['query_id'] == query_id]
        if ground_truth_row.empty:
            ndcg_por_query[query_id] = 0.0
            continue       
        relevancia_docs = ground_truth_row['doc_ids'].values[0]
        relevancias_reales = ground_truth_row['relevancias'].values[0]

        M = len(relevancia_docs)      
        relevancias_obtenidas = [relevancias_reales[relevancia_docs.index(doc)] if doc in relevancia_docs else 0 for doc in doc_retrieved[:M]]
        dcg = calcular_dcg(relevancias_obtenidas, M)
        relevancias_ideales = sorted(relevancias_reales, reverse=True)
        idcg = calcular_dcg(relevancias_ideales, M)
        ndcg_p_m = dcg / idcg if idcg > 0 else 0.0
        ndcg_por_query[query_id] = ndcg_p_m
    
    return ndcg_por_query

ndcg_resultados = calcular_ndcg_p_m(resultados_df, ground_truth_df)
ndcgDF = pd.DataFrame(list(ndcg_resultados.items()), columns=['query_id', 'ndcg'])
ndcgDF

,query_id,ndcg
0,q01,0.000000
1,q02,0.074080
2,q03,0.947914
3,q04,0.000000
4,q06,0.084732
5,q07,0.000000
6,q08,0.148853
7,q09,0.117489
8,q10,0.000000
9,q12,0.000000


# Calculo de la metrica de evaluación 𝑀𝐴𝑃
Finalmente, se realiza el procesamiento de la información para aplicar la metrica de evaluación 𝑀𝐴𝑃 (Mean Average Precision), la métrica mide la precisión promedio en todas las consultas. Se calcula obteniendo la precisión en cada posición relevante de una consulta, promediando esas precisiones, y luego promediando los resultados para todas las consultas. Representa qué tan bien ordenados están los documentos relevantes dentro de los resultados. promediando las precisiones acumuladas de todas las consultas, devolviendo una métrica global de precisión, mostrando el resultado final general de evaluación del ejercicio.

In [23]:
# Calculo de MAP

def calcular_precision_acumulada(relevance_list):
    precisiones = []
    num_relevant = 0
    for i, relevancia in enumerate(relevance_list):
        if relevancia == 1:
            num_relevant += 1
            precisiones.append(num_relevant / (i + 1))  
    if precisiones:
        return sum(precisiones) / len(precisiones)
    else:
        return 0.0

def calcular_map(resultados_df, ground_truth_df):
    average_precisions = []
    if 'doc_ids' not in resultados_df.columns:
        resultados_df['doc_ids'] = resultados_df['docs'].apply(lambda x: [doc.split(':')[0] for doc in x.split(',')]) 
    resultados_df['doc_ids'] = resultados_df['doc_ids'].apply(limpiar_identificadores)
    if 'doc_ids' not in ground_truth_df.columns:
        ground_truth_df['doc_ids'] = ground_truth_df['doc_id_relevance'].apply(lambda x: [doc.split(':')[0] for doc in x.split(',')])
    ground_truth_df['relevancias'] = ground_truth_df['doc_id_relevance'].apply(lambda x: [int(doc.split(':')[1]) for doc in x.split(',')])
    for _, row in resultados_df.iterrows():
        query_id = row['query_id']
        doc_retrieved = row['doc_ids']  
        ground_truth_row = ground_truth_df[ground_truth_df['query_id'] == query_id]
        if ground_truth_row.empty:
            average_precisions.append(0.0)
            continue
        relevancia_docs = ground_truth_row['doc_ids'].values[0]
        relevance_list = [1 if doc in relevancia_docs else 0 for doc in doc_retrieved]
        avg_precision = calcular_precision_acumulada(relevance_list)
        average_precisions.append(avg_precision)
    map_score = sum(average_precisions) / len(average_precisions) if average_precisions else 0.0
    return map_score

# Ejemplo de uso
map_score = calcular_map(resultados_df, ground_truth_df)
print("\nMAP (Mean Average Precision):", map_score)


MAP (Mean Average Precision): 0.14663681501171802
